<a href="https://colab.research.google.com/github/poet-developer/2024WebScrapping/blob/main/RISS_%ED%95%99%EC%9C%84_%EB%85%BC%EB%AC%B8_%EC%84%9C%EC%A7%80%EC%A0%95%EB%B3%B4_%EC%8A%A4%ED%81%AC%EB%9E%98%ED%95%91_%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import math
import pickle
import time
import sys
import pandas as pd
from tqdm.notebook import tqdm
from random import uniform
import lxml
import lxml.etree as et
from bs4 import BeautifulSoup
import glob
import urllib3
urllib3.disable_warnings()
from natsort import natsorted
import re
import concurrent.futures
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse #url parse 모듈

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "DNT": "1", # Do Not Track 요청 헤더 (사용자 추적 거부)
    # 필요한 추가 헤더를 여기에 추가
}

In [4]:
def request_until_success(url, headers, timeout=7, delay=3):
    """
    Continuously makes a request to the specified URL with a timeout until the request is successful.

    Args:
    - url (str): The URL to request.
    - timeout (int): The timeout for the request in seconds.
    - delay (int): The delay between retries in seconds.

    Returns:
    - response: The successful response from the server.
    """
    attempt = 1
    while True:
        try:
            # print(f"Attempt {attempt}")
            response = requests.get(url,headers=headers, timeout=timeout)
            response.raise_for_status()  # Raises HTTPError for bad responses
            # print("Request successful")
            return response
        except requests.Timeout as e:
            print(f"Request timed out: {e}")
        except requests.RequestException as e:
            print(f"Request failed: {e}")

        # print(f"Retrying in {delay} seconds...")
        time.sleep(delay)
        attempt += 1

# 검색 기준
* 학과: 디지털
* 논문명: 디지털큐레이션
* 년도: 2000 ~ 2024
* 국내 박사


In [5]:
url = "https://riss.kr/search/Search.do?isDetailSearch=Y&searchGubun=true&viewYn=OP&sflag=1&fsearchMethod=searchDetail&isFDetailSearch=N&colName=bib_t&field1=znTitle&keyword1=%EB%94%94%EC%A7%80%ED%84%B8%ED%81%90%EB%A0%88%EC%9D%B4%EC%85%98&op1=OR&field2=znMajor&keyword2=%EB%94%94%EC%A7%80%ED%84%B8&op2=AND&field3=znSubject&keyword3=&op3=AND&field4=znAbstract&keyword4=&op4=AND&field5=znKtoc&keyword5=&p_year1=2000&p_year2=2024&mat_subtype=T2#redirect"

In [6]:
req = request_until_success(url,headers).text

In [7]:
soup = BeautifulSoup(req, 'lxml')

In [12]:
#html스크래핑 class,elem 사용, for 구문 사용
cont_ml60_classes = soup.find_all('div', class_='cont ml60')

data = [{
    'title': cont_ml60.find('p', class_='title').get_text(strip=True) if cont_ml60.find('p', class_='title') else None,
    'link': cont_ml60.find('p', class_='title').find('a').get('href') if cont_ml60.find('p', class_='title') and cont_ml60.find('p', class_='title').find('a') else None,
    'writer': cont_ml60.find('p', class_='etc').find_all('span')[0].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 0 else None,
    'assigned': cont_ml60.find('p', class_='etc').find_all('span')[1].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 1 else None,
    'year': cont_ml60.find('p', class_='etc').find_all('span')[2].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 2 else None,
    'grad': cont_ml60.find('p', class_='etc').find_all('span')[3].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 3 else None,
    'preAbstract': cont_ml60.find('p', class_='preAbstract').get_text(strip=True) if cont_ml60.find('p', class_='preAbstract') else None
} for cont_ml60 in cont_ml60_classes]

In [44]:
data

Request failed: HTTPSConnectionPool(host='www.riss.krhttps', port=443): Max retries exceeded with url: //www.riss.krhttps://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=973f266e1f85775affe0bdc3ef48d419&keyword= (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7b584b9fb6d0>: Failed to resolve 'www.riss.krhttps' ([Errno -2] Name or service not known)"))
Request failed: HTTPSConnectionPool(host='www.riss.krhttps', port=443): Max retries exceeded with url: //www.riss.krhttps://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=bbe6ac903ed9a305ffe0bdc3ef48d419&keyword= (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7b584b9baec0>: Failed to resolve 'www.riss.krhttps' ([Errno -2] Name or service not known)"))
Request failed: HTTPSConnectionPool(host='www.riss.krhttps', port=443): Max retries exceeded with url: //www.riss.krhttps://www.riss.kr/search/detail/DetailView.d

[{'title': '디지털 큐레이션 서비스의 프레임워크 개발에 관한 연구',
  'link': '/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=cc4e874732ed71cfffe0bdc3ef48d419&keyword=',
  'writer': '안영희',
  'assigned': '忠南大學校 大學院',
  'year': '2011',
  'grad': '국내박사',
  'preAbstract': 'The purpose of this study is to establish a digital curation service for the creation, management, maintenance, sharing and reuse of digital resources. Specifically, this study is aimed at examining  digital curation and to developing a framework for its practical application.\n\nThis study has defined digital curation as ‘General preservation activities including digital preservation, digital archiving, and data curation.’ Thus the definition adds value to digital information resources so that current and future users can both access and contribute to new information and knowledge. \n\nThe life cycle of digital curation in this study originated from cases of AHDS, DCC, DCC&U, and LIFE 1, and the policies of digital curatio

In [14]:
# 검색된 논문 개수
num_span = soup.find('span', class_='num')

if num_span is not None:
    total_count = int(num_span.get_text().replace(',', ''))
else:
    total_count = 0

total_count

590

In [33]:
# The base URL without the 'iStartCount' parameter
parsed_url = urlparse(url) #url을 추출가능한 형태로 parsing, 데이터타입은 딕셔너리

params = parse_qs(parsed_url.query) #url.query추출 # 쿼리 문자열을 파이썬 데이터 구조로 구문 분석하기 위해 사용.
# Remove the 'iStartCount' parameter
params.pop('iStartCount', None)

# Re-encode the query parameters
new_query = urlencode(params, doseq=True)

# Replace the old query parameters with the new ones
new_url = parsed_url._replace(query=new_query)

# Unparse the URL #스크래핑을 위한 새로운 url 만들기
base_url = urlunparse(new_url)

In [16]:
# The base URL without the 'iStartCount' parameter
# base_url = "https://www.riss.kr/search/Search.do?isDetailSearch=Y&searchGubun=true&viewYn=OP&queryText=znMajor,국어국문@op,OR@znMajor,국문&strQuery=&exQuery=&exQueryText=&order=/DESC&onHanja=false&strSort=RANK&p_year1=2000&p_year2=2023&orderBy=&mat_type=&mat_subtype=T2&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&db_type=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=bib_t&colName=bib_t&pageScale=100&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query="

# Calculate the number of iterations
iterations = math.ceil(total_count / 100)

# Initialize an empty list to store the data
data = []

def fetch_data(i):
    # Create the URL with the incremented 'iStartCount' parameter
    url = f"{base_url}&iStartCount={i * 100}"

    # Send a GET request to the URL
    response = request_until_success(url, headers)

    # Create a BeautifulSoup object from the response text
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all 'div' elements with class 'cont ml60'
    cont_ml60_classes = soup.find_all('div', class_='cont ml60')

    # Extract the data and return it
    return [{
        'title': cont_ml60.find('p', class_='title').get_text(strip=True) if cont_ml60.find('p', class_='title') else None,
        'link': cont_ml60.find('p', class_='title').find('a').get('href') if cont_ml60.find('p', class_='title') and cont_ml60.find('p', class_='title').find('a') else None,
        'writer': cont_ml60.find('p', class_='etc').find_all('span')[0].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 0 else None,
        'assigned': cont_ml60.find('p', class_='etc').find_all('span')[1].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 1 else None,
        'year': cont_ml60.find('p', class_='etc').find_all('span')[2].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 2 else None,
        'grad': cont_ml60.find('p', class_='etc').find_all('span')[3].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 3 else None,
        'preAbstract': cont_ml60.find('p', class_='preAbstract').get_text(strip=True) if cont_ml60.find('p', class_='preAbstract') else None
    } for cont_ml60 in cont_ml60_classes]

# Loop over the range, incrementing by 100 each time #느린 작업을 cpu사용을 늘려 속도를 올림(multi)
with concurrent.futures.ThreadPoolExecutor() as executor:
    data_list = list(tqdm(executor.map(fetch_data, range(iterations)), total=iterations))

# Flatten the list of lists
data = [item for sublist in data_list for item in sublist]

  0%|          | 0/6 [00:00<?, ?it/s]

In [17]:
df = pd.DataFrame(data)
df

,title,link,writer,assigned,year,grad,preAbstract
0,디지털 큐레이션 서비스의 프레임워크 개발에 관한 연구,/search/detail/DetailView.do?p_mat_type=be54d9...,안영희,忠南大學校 大學院,2011,국내박사,The purpose of this study is to establish a di...
1,기록문화유산의 디지털 큐레이션 모델 연구 : 국채보상운동 기록물을 중심으로,/search/detail/DetailView.do?p_mat_type=be54d9...,김지명,韓國學中央硏究院,2017,국내박사,The purpose of this dissertation is to develop...
2,재사용과 공유를 위한 수업자료 디지털 큐레이션 시스템 설계에 대한 연구,/search/detail/DetailView.do?p_mat_type=be54d9...,박양하,연세대학교 대학원,2015,국내박사,None
3,모바일 포렌식 관점의 디지털 증거능력 수준 평가방법 연구 : 전문가 인터뷰와 AHP...,/search/detail/DetailView.do?p_mat_type=be54d9...,김영준,아주대학교,2022,국내박사,정보통신기술의 비약적인 발전과 함께 최근 다양한 정보기기들의 대중화로 기존의 물리적...
4,디지털 캐릭터의 UV(Uncanny Valley) 현상에 관한 연구 : 리얼리즘 기...,/search/detail/DetailView.do?p_mat_type=be54d9...,김남훈,경성대학교 디지털디자인전문대학원,2013,국내박사,최근 CG 기반 디지털 컨텐츠 및 예술작품을 둘러싸고 벌어지는 가장 뜨거운 논의 중...
5,"MATLAB을 이용한 디지털 이미지의 위, 변조 분석 연구",/search/detail/DetailView.do?p_mat_type=be54d9...,이정범,중앙대학교,2007,국내박사,디지털 이미지 시스템의 개발과 함께 기존의 아날로그 시스템보다 우수한 품질의 이미지...
6,힐링콘텐츠 개념을 적용한 정신치료용 힐링게임 연구,/search/detail/DetailView.do?p_mat_type=be54d9...,이연숙,국민대학교,2007,국내박사,문화콘텐츠산업은 디지털과 인터넷 기술의 발전과 더불어 산업의 전통적인 패러다임을 변...
7,디지털게임 보상시스템의 지수함수 메커니즘을 응용한 디지털 페다고지 모형 연구,/search/detail/DetailView.do?p_mat_type=be54d9...,문만기,중앙대학교 첨단영상대학원,2010,국내박사,디지털 패러다임이 보편화되면서 지금까지 단순한 놀이로 간주되고 있었던 디지털게임에 ...
8,리눅스 시스템 포렌식을 위한 증거추출 및 분석모듈,/search/detail/DetailView.do?p_mat_type=be54d9...,송유진,한서대학교 대학원,2012,국내박사,There is a wide range of cyber related crimes ...
9,군(軍) 위기 시 커뮤니케이션 전략 연구 : 디지털 이미지를 활용한 메시지 효과를 ...,/search/detail/DetailView.do?p_mat_type=be54d9...,김진수,상명대학교 일반대학원,2022,국내박사,Maintaining reputation and restoring image in ...


In [45]:
# csv로 저장
df.to_csv('riss_title.csv', index=False)

# 세부내용 추출

In [38]:
df['link'] = 'https://www.riss.kr' + df['link'] #특정페이지 들어가는 url 확보. df는 데이터표. 데이터표에 link라는 열이름 값에 대해 string데이터 추가.

In [39]:
headers_for_riss = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7,ja;q=0.6",
    "Connection": "keep-alive",
    "Cookie": "Your Cookie Here",
    "Host": "www.riss.kr",
    "Referer": None,
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "sec-ch-ua": '"Google Chrome";v="123", "Not:A-Brand";v="8", "Chromium";v="123"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "Windows"
}

In [40]:
def fetch_data(link):
    headers_for_riss["Referer"] = link
    # response = requests.get(link, headers=headers)
    response = request_until_success(link, headers_for_riss)
    soup = BeautifulSoup(response.text, 'lxml')
    data = {}
    data['link'] = link
    for li in soup.find('div',class_='infoDetailL').find_all('li'):
        key_element = li.find('span', {'class': 'strong'})
        value_element = li.find('div')
        if key_element and value_element:
            key = key_element.text.strip()
            value = value_element.text.strip()
            data[key] = value
    try:
        text_off_divs = soup.find('div', class_='content additionalInfo').find_all('div', class_='text off')
        title_text_dict = {}
        for div in text_off_divs:
            title = div.find_previous_sibling('p', class_='title')
            if title:
                if title.text.strip() == "참고문헌 (Reference)":
                    title_text_dict[title.text.strip()] = [p.text.strip() for p in div.find_all('p')]
                else:
                    title_text_dict[title.text.strip()] = div.text.strip()
        data.update(title_text_dict)
    except:
        print(f'참고문헌 등 추가 정보 없음 :{link}')
    return data

In [43]:
# 실행 코드
links = df['link'].tolist()[:10] # 100개만 테스트 다하려면 [:100] 제거
with concurrent.futures.ThreadPoolExecutor() as executor:
    data_list = list(tqdm(executor.map(fetch_data, links), total=len(links)))

Request failed: HTTPSConnectionPool(host='www.riss.krhttps', port=443): Max retries exceeded with url: //www.riss.krhttps://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=973f266e1f85775affe0bdc3ef48d419&keyword= (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7b584a87e560>: Failed to resolve 'www.riss.krhttps' ([Errno -2] Name or service not known)"))
Request failed: HTTPSConnectionPool(host='www.riss.krhttps', port=443): Max retries exceeded with url: //www.riss.krhttps://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=23fa2219e5c7be98ffe0bdc3ef48d419&keyword= (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7b584b994f70>: Failed to resolve 'www.riss.krhttps' ([Errno -2] Name or service not known)"))Request failed: HTTPSConnectionPool(host='www.riss.krhttps', port=443): Max retries exceeded with url: //www.riss.krhttps://www.riss.kr/search/detail/DetailView.do

  0%|          | 0/10 [00:00<?, ?it/s]

Request failed: HTTPSConnectionPool(host='www.riss.krhttps', port=443): Max retries exceeded with url: //www.riss.krhttps://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=8aeb7ca8f687fca8ffe0bdc3ef48d419&keyword= (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7b584b9f9bd0>: Failed to resolve 'www.riss.krhttps' ([Errno -2] Name or service not known)"))
Request failed: HTTPSConnectionPool(host='www.riss.krhttps', port=443): Max retries exceeded with url: //www.riss.krhttps://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=75fffc1e252963b5ffe0bdc3ef48d419&keyword= (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7b584a87c640>: Failed to resolve 'www.riss.krhttps' ([Errno -2] Name or service not known)"))Request failed: HTTPSConnectionPool(host='www.riss.krhttps', port=443): Max retries exceeded with url: //www.riss.krhttps://www.riss.kr/search/detail/DetailView.do

KeyboardInterrupt: 

Request failed: HTTPSConnectionPool(host='www.riss.krhttps', port=443): Max retries exceeded with url: //www.riss.krhttps://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=973f266e1f85775affe0bdc3ef48d419&keyword= (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7b584b9f9f30>: Failed to resolve 'www.riss.krhttps' ([Errno -2] Name or service not known)"))Request failed: HTTPSConnectionPool(host='www.riss.krhttps', port=443): Max retries exceeded with url: //www.riss.krhttps://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=cc4e874732ed71cfffe0bdc3ef48d419&keyword= (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7b584a87ea70>: Failed to resolve 'www.riss.krhttps' ([Errno -2] Name or service not known)"))

Request failed: HTTPSConnectionPool(host='www.riss.krhttps', port=443): Max retries exceeded with url: //www.riss.krhttps://www.riss.kr/search/detail/DetailView.d

In [ ]:
detail = pd.DataFrame(data_list)
detail

In [ ]:
# csv로 저장
detail.to_csv('riss_detail.csv', index=False)